## Analyst Tutorial: Ancestry Comparisong using PCA

This Jupyter Notebook documents the analysis of ancestry comparison using Principal Component Analysis (PCA). The analysis is performed on a dataset generated by the Large Cohort Pipeline, which processed raw FASTQ files and generated PCA values. Additionally, we will compare the ancestry of our dataset to a public dataset by calculating PCA values for the public dataset.

### Background
The Large Cohort Pipeline is a powerful tool for processing genomic data and extracting valuable insights. By submitting raw FASTQ files to the pipeline, we obtained PCA values that provide information about the genetic ancestry of our dataset. To gain further insights and compare our dataset with a public dataset, we will calculate PCA values for the public dataset as well.

### Objectives
The main objectives of this analysis are as follows:
1. Compare the genetic ancestry of our dataset with the public dataset using PCA.
2. Visualize the PCA results to identify patterns and similarities/differences between the datasets.
3. Gain insights into the genetic diversity and population structure of the datasets.
4. Gain familiarity with using Hail to conduct analyses of large datasets

### Methodology
We will the Hail for genomic data analysis, including PCA calculation and visualisation. The notebook consists of multiple cells, each containing code snippets and markdown cells for documentation.

### Conclusion
By comparing the PCA values of our dataset with the public dataset, we aim to gain insights into the genetic ancestry and population structure of both datasets. The analysis will provide valuable information for understanding the genetic diversity and relationships between different populations.

Let's proceed with the analysis and explore the fascinating world of genetic ancestry comparison using PCA!

### Hail Background
**Hail**: Hail is an open-source, scalable framework for exploring and analyzing genomic data. It's built to handle large-scale genomic data and provides tools for data manipulation, statistical analysis, and visualisation. Hail is particularly powerful for genome-wide association studies (GWAS) and other genetic analyses.

**Hail Tables**: Hail Tables are one of the two primary data structures in Hail. They are similar to SQL tables, R data.frames, or pandas DataFrames. A Hail Table is a distributed dataset, that can be used to hold various types of data, including numerical, categorical, and genomic data. Hail Tables support a variety of operations like filtering, aggregation, and joins.

**Hail MatrixTables**: MatrixTables are the second primary data structure in Hail and are designed to hold large-scale, distributed genomic data. A MatrixTable consists of a two-dimensional matrix of entries, indexed by row keys (representing variant data) and column keys (representing sample data). Each cell in the matrix represents the genetic data for a specific variant and sample. Along with the matrix, there are also row, column, and global fields for storing related data. MatrixTables support a variety of operations like filtering, aggregation, linear algebra operations, and joins.

Hail provides tutorials specifically designed to introduce individuals to working with Tables and MatrixTables:
- [Table tutorial](https://hail.is/docs/0.2/tutorials/03-tables.html)
- [MatrixTable tutorial](https://hail.is/docs/0.2/tutorials/07-matrixtable.html)

Additionally, [here](https://hail.is/docs/0.2/tutorials-landing.html) is Hail's complete list of tutorials that provide users with the opportunity to explore more manipulations using their data structures

![Hail Matrix Table Diagram](Hail_matrix_table_diagram.png)
Source: https://hail.is/docs/0.2/_static/cheatsheets/hail_matrix_tables_cheat_sheet.pdf

## Imports and Reading in Data from GCP

Before starting, we need to import the necessary packages. If you have followed the notebook setup properly, Hail should already be installed along with Python and some basic packages, and can be imported directly.

In [1]:
import hail as hl
import pprint
from hail.plot import show
hl.plot.output_notebook()

Loading BokehJS ...

Loading BokehJS ...

This notebook should be set up inside the bucket that contains the data we are wanting to analyse, this way we have permissions to access the data. In the case of this tutorial it is the `bioheart` bucket. While some buckets/projects are capable of reading the contents of other buckets, not all of them are, so it is best to work from a notebook that is within the bucket containing our data.

#### Reading in Data
We will now be reading in our data. If you remember in our config file we set the parameter `output_version` to be `5.0`. This has created a directory within the `large_cohort` directory of the `cpg-bioheart-test` bucket.
While we have mentioned `Tables` (`.ht`) and `MatrixTables` (`.mt`) above, we will also be working with `Variant Dataset` (`.vds`) files. 
- `vds` or `Variant Dataset` ([documentation](https://hail.is/docs/0.2/vds/index.html))
    - Contains variant information for every sample in the cohort
    - Briefly; because we are dealing with cohort-level data there will be regions of genome that, for every sample in the cohort, is homozygous reference, as well as information where sample(s) are non-reference. To maintain both reference and non-reference information in a scalable manner, a `.vds` file is made up of two `MatrixTables`, the `reference_data` and `variant_data` `MatrixTable`'s.
- `sample_qc_ht`: this Hail Table is generated by the `SampleQC` stage of the `large cohort` pipeline. It contains the sex of each sample in the cohort and PCA values for each sample to be used in ancestry comparison as well as quality metrics.
- `relateds_to_drop_ht`: this Hail Table is generated by the `Relatedness` stage of the `large cohort` pipeline. It contains information about related samples in the cohort and which samples, if related, should be dropped from the cohort. For samples that are related, they are ranked according to their sample QC metrics and the sample with the worst metrics is dropped from the cohort so that there is only one sample left per family. See code [here](https://github.com/populationgenomics/production-pipelines/blob/824019c6eb0387d10aff047145e92583cd3e701c/cpg_workflows/large_cohort/relatedness.py#L78)
- `pop_ht`: a Hail Table containing the assigned global population labels. Hail uses a Random Forest machine learning model based on PCA scores to assign global population labels to samples. See code [here](https://github.com/populationgenomics/production-pipelines/blob/824019c6eb0387d10aff047145e92583cd3e701c/cpg_workflows/large_cohort/ancestry_pca.py#L215). Population label is assigned based on previously known ancestry of the HGDP and 1KG datasets (calculated from gnomAD).
- `vqsr.ht`: Hail Table resulting from the Variant Quality Score Recalibration process. This table contains information about the variants that were filtered out during the VQSR process.
- `freq_ht`: Hail Table containing information about the allele frequency of each variant in the cohort. This table is generated by the `VariantQC` stage of the `large cohort` pipeline. Generates frequency annotations of AF, AC, AN, InbreedingCoeff
    - AF (Allele Frequency): This is the proportion of all alleles at a given location in a population that are of a particular type
    - AC (Allele Count): This is the total number of times a specific allele appears in a population.
    - AN (Allele Number): This is the total number of alleles at a given location in a population.
    - InbreedingCoeff: This is a measure of the likelihood of genetic traits in a population being inherited due to the parents being closely related.
- `dense_mt`: Hail Matrix Table containing the genotype information for each sample in the cohort. This table is generated by the `Genotype` and subsequent `GenotypeQC` stages of the `large cohort` pipeline. This table contains the genotype information for a set of pre-determined QC sites (according to `gs://cpg-common-main/references/gnomad/v0/sample_qc/pre_ld_pruning_qc_variants.ht`) for each sample in the cohort. The table is dense, meaning that it contains all variants for all samples in the cohort.

In [2]:
vds = hl.vds.read_vds('gs://cpg-bioheart-test/vds/5-0.vds/')
sample_qc_ht = hl.read_table('gs://cpg-bioheart-test/large_cohort/5-0/sample_qc.ht/')
relateds_to_drop_ht = hl.read_table('gs://cpg-bioheart-test/large_cohort/5-0/relateds_to_drop.ht/')
pop_ht = hl.read_table('gs://cpg-bioheart-test/large_cohort/5-0/ancestry/inferred_pop.ht/')
vqsr_ht = hl.read_table('gs://cpg-bioheart-test/large_cohort/5-0/vqsr.ht/')
freq_ht = hl.read_table('gs://cpg-bioheart-test/large_cohort/5-0/frequencies.ht/')
dense_mt = hl.read_matrix_table('gs://cpg-bioheart-test/large_cohort/5-0/dense_subset.mt/')

Initializing Hail with default parameters...
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/opt/conda/envs/python310/lib/python3.10/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://dd7b7b83e829:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.126-ee77707f4fab
LOGGING: writing to /home/jupyter/hail-20240104-2101-0.2.126-ee77707f4fab.log


### Pre-processing and Filtering

As stated previously, a `hl.vds` object is a combination of two `MatrixTables`; `vds.reference_data` and `vds.variant_data`, which are both sparse representations of the data. `vds.reference_data` contains reference data as 'blocks' of information, meaning that reference data is not explicitly stored at each variant site but is represented in intervals ('blocks') where it is necessary. `vds.variant_data` is a sparse matrix of non-reference calls. Our analysis requires a dense representation of the data, meaning each position in the genome is represented in the data i.e. reference data is explicitly stored at each position and not in 'blocks', as well as non-reference calls being present. To achieve this, we will use the `to_dense_mt()` function to convert our `vds` into a `MatrixTable` with a dense representation of the data.

Additionally, we use the `split_multi()` to split multi-allelic variants into bi-allelic variants. After splitting, each row represents a single variant.

In [3]:
# Row-level tables require a split+dense matrix table:
vds = hl.vds.split_multi(vds, filter_changed_loci=True)
mt = hl.vds.to_dense_mt(vds)

#### Sample vs Row level filtering
We will now filter our `MatrixTable` according to the outputs of the pipeline. 
Any filtering that works using the column key is sample-level filtering and thus will be filtering on sample-level metrics. 
For example: any samples that did not pass the `SampleQC` stage of the pipeline will be removed from the `MatrixTable` if they are not in the `sample_qc_ht` table. This is sample-level filtering because we are filtering on the sample-level metrics in the `sample_qc_ht` table.
`vqsr.ht` contains information about the variants that were filtered out during the VQSR process. We will use this table to filter out variants that did not pass the VQSR process. This is row-level filtering because we are filtering on the variant-level metrics in the `vqsr.ht` table.

In [4]:
# Hard-filtering samples and variants:
mt = mt.filter_cols(hl.len(sample_qc_ht[mt.col_key].filters) > 0, keep=False) # sample-level filtering
mt = mt.filter_cols(hl.is_defined(relateds_to_drop_ht[mt.col_key]), keep=False) # sample-level filtering
mt = mt.filter_rows(hl.len(vqsr_ht[mt.row_key].filters) > 0, keep=False) # row-level filtering

#### Annotation

Annotating our `MatrixTable` means adding `column fields` to the `MatrixTable`'s `columns` `Hail Table` that contain information about the samples and, vice versa, adding new `row fields` to the `rows` `Hail Table` that contains additional information about the calls. We will annotate our `MatrixTable` with the following information:
- Sample QC metrics from the `sample_qc_ht` table
- Sample population label from the `pop_ht` table
- Call information from the `vqsr` table
- Call frequency information from the `freq_ht` table

In [5]:
# Annotating samples and variants:
mt = mt.annotate_cols(**sample_qc_ht[mt.col_key])
mt = mt.rename({'filters' : 'sample_qc_filters'}) # to avoid conflict with vqsr column labels
mt = mt.annotate_cols(**pop_ht[mt.col_key])
mt = mt.annotate_rows(**vqsr_ht[mt.row_key])
mt = mt.annotate_rows(**freq_ht[mt.row_key])

In [6]:
# import annotations table with ancestry information
# even though we have an 'ancestry' output from the pipeline, we won't use it
table = (hl.import_table('annotations.txt', impute=True).key_by('Sample'))

2024-01-04 21:02:29.088 Hail: INFO: Reading table to impute column types
2024-01-04 21:02:30.624 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'population' as type str (imputed)
  Loading field 'superpopulation' as type str (imputed)


In [8]:
# annotate matrix table columns with ancestry information
mt = mt.annotate_cols(superpopulation=table[mt.s].superpopulation, 
                      population=table[mt.s].population)

In [9]:
# plot ancestry
p = hl.plot.scatter(mt.pca_scores[0],
                    mt.pca_scores[1],
                    label=mt.superpopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

2024-01-04 21:02:55.654 Hail: WARN: aggregate_cols(): Aggregates over cols ordered by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


## Conducting PCA on Public Dataset

In this section, we will be conducting a Principal Component Analysis (PCA) using Hail on the publicly available Human Genome Diversity Project - 1000 Genomes Project (HGDP-1KG) dataset. As we've seen, PCA ican be used to identify and visualise genetic variation between individuals. It can help us understand the genetic structure of our dataset and identify potential population stratification, which is crucial for many genetic analyses.

The HGDP-1KG dataset is a comprehensive collection of human genetic variation from globally diverse populations. It combines data from two significant projects: the Human Genome Diversity Project (HGDP) and the 1000 Genomes Project (1KG). This dataset is readily accessible through Hail's interface as it is hosted by `gnomAD`.

Our goal in this section is to compare the ancestry of our dataset with the ancestry of the HGDP-1KG dataset. By projecting our samples onto the genetic variation patterns identified in the HGDP-1KG dataset, we can see where our samples fit in the global population structure. This can provide valuable insights into the genetic background of our samples and inform subsequent analyses.

Let's get started with the PCA.

In [10]:
hl.utils.get_1kg('data/')

2024-01-04 21:14:29.256 Hail: INFO: downloading 1KG VCF ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz
2024-01-04 21:14:31.451 Hail: INFO: importing VCF and writing to matrix table...
2024-01-04 21:14:32.133 Hail: INFO: scanning VCF for sortedness...
2024-01-04 21:14:35.146 Hail: INFO: Coerced sorted VCF - no additional import work to do
2024-01-04 21:14:40.409 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 16 partitions to data/1kg.mt
2024-01-04 21:14:40.510 Hail: INFO: downloading 1KG annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt
2024-01-04 21:14:41.056 Hail: INFO: downloading Ensembl gene annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/ensembl_gene_annotations.txt
2024-01-04 21:14:42.298 Hail: INFO: Done!


In [12]:
mt_1kg = hl.read_matrix_table('data/1kg.mt')

In [21]:
table_1kg = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
mt_1kg = mt_1kg.annotate_cols(pheno = table_1kg[mt_1kg.s])

2024-01-04 21:25:13.742 Hail: INFO: Reading table to impute column types
2024-01-04 21:25:14.248 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)


In [14]:
# Quality control
mt_1kg = hl.sample_qc(mt_1kg)
mt_1kg = mt_1kg.filter_cols((mt_1kg.sample_qc.dp_stats.mean >= 4) & (mt_1kg.sample_qc.call_rate >= 0.97))

ab = mt_1kg.AD[1] / hl.sum(mt_1kg.AD)

filter_condition_ab = ((mt_1kg.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt_1kg.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt_1kg.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt_1kg.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt_1kg = mt_1kg.filter_entries(filter_condition_ab)

mt_1kg = hl.variant_qc(mt_1kg)

mt_1kg = mt_1kg.filter_rows(mt_1kg.variant_qc.AF[1] > 0.01)
mt_1kg = mt_1kg.filter_rows(mt_1kg.variant_qc.p_value_hwe > 1e-6)

Filtering 3.60% entries out of downstream analysis.


In [ ]:
# Calculate PCA scores
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_1kg.GT)
mt_1kg = mt_1kg.annotate_cols(scores = pcs[mt_1kg.s].scores)

In [22]:
p = hl.plot.scatter(mt_1kg.scores[0],
                    mt_1kg.scores[1],
                    label=mt_1kg.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)